In [1]:
from collections import Counter
from datetime import datetime
import numpy as np
import os
import pandas as pd
import pytz
import re
import time
import tweepy

In [2]:
# auth = tweepy.AppAuthHandler(user_secrets.get_secret("ulWFXaeuShn1uBIyzKjkOx49D"), user_secrets.get_secret("rfctAYM4lJSP1yEBih5yA71nekWkZvqQE9vCsNTfLec5Mp4OW9"))
# api = tweepy.API(auth)

In [3]:
consumer_key = "***************"
consumer_secret = "***********"
access_token = "**************"
access_token_secret = "*************"

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [5]:
wordle_start = pytz.timezone("US/Pacific").localize(datetime(2021, 6, 19))
now = pytz.utc.localize(datetime.now()).astimezone(pytz.timezone("US/Pacific"))

wordle_id = (now-wordle_start).days
print("Today's wordle_id is: %d (%s)" % (wordle_id, now.strftime("%Y-%m-%d %H:%M PT")))

Today's wordle_id is: 302 (2022-04-17 15:07 PT)


In [6]:
wordle_tweets = []
try:
    cursor = tweepy.Cursor(api.search_tweets, q="wordle %d" % wordle_id)
    tweets = list(cursor.items(500))
    for tweet in tweets:
        wordle_tweets.append((wordle_id, tweet.id, tweet.created_at, tweet.author.screen_name, tweet.text))

    print("Pulled %d tweets for wordle %d" % (len(wordle_tweets), wordle_id))

except tweepy.RateLimitError:
    handle_rate_limit_error()

Pulled 500 tweets for wordle 302


In [7]:
def is_valid_wordle_tweet(tweet, wordle_id): 
    text = (tweet.replace("Y", "y").replace("🟩", "Y")
                 .replace("M", "m").replace("🟨", "M")
                 .replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
    
    for i in range(wordle_id-20, wordle_id+20):
        if i==wordle_id:
            continue
        if str(i) in text:
            return False
    
    if len(re.findall("Wordle %d" % wordle_id, text)) != 1:
        return False

    if re.match("Wordle %d [2-6]/6\n\n[YMN]{5}\n" % wordle_id, text) is None:
        return False
    return True     

In [8]:
new_tweets_df = pd.DataFrame([tweet for tweet in wordle_tweets if is_valid_wordle_tweet(tweet[4], tweet[0])],
                             columns=["wordle_id", "tweet_id", "tweet_date", "tweet_username", "tweet_text"])
new_tweets_df

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
0,302,1515874486247170049,2022-04-18 02:06:55+00:00,AlyssaBFrey,Wordle 302 2/6\n\n🟩⬛⬛🟩🟩\n🟩🟩🟩🟩🟩
1,302,1515874474058465282,2022-04-18 02:06:52+00:00,MrCitroen,Wordle 302 3/6\n\n🟩⬜⬜🟨⬜\n🟩⬜🟩🟩🟩\n🟩🟩🟩🟩🟩
2,302,1515874437282910210,2022-04-18 02:06:44+00:00,m3tamonk,Wordle 302 3/6\n\n⬛🟨⬛🟩⬛\n🟨⬛⬛⬛🟩\n🟩🟩🟩🟩🟩
3,302,1515874398426779650,2022-04-18 02:06:34+00:00,octavianasr,Wordle 302 5/6\n\n⬛⬛🟨🟨🟨\n🟨🟨🟨⬛⬛\n⬛🟨🟨🟨⬛\n🟨🟨🟩🟩🟩\n...
4,302,1515874377346207744,2022-04-18 02:06:29+00:00,talbs,Wordle 302 4/6\n\n⬜⬜🟨🟨🟨\n⬜🟨🟨🟨🟩\n🟨🟨🟩🟩🟩\n🟩🟩🟩🟩🟩
...,...,...,...,...,...
368,302,1515868281172865036,2022-04-18 01:42:16+00:00,lover_music3,Wordle 302 5/6\n\n⬛⬛⬛⬛⬛\n🟦⬛⬛🟦⬛\n⬛🟦🟦⬛⬛\n🟦🟦🟧🟧🟧\n...
369,302,1515868278027177993,2022-04-18 01:42:15+00:00,jwcarroll,Wordle 302 2/6\n\n⬜⬜🟨⬜🟩\n🟩🟩🟩🟩🟩
370,302,1515868247886901250,2022-04-18 01:42:08+00:00,l0v3LynnMarie,Wordle 302 5/6\n\n⬜⬜🟨⬜⬜\n⬜🟨🟨⬜⬜\n🟨🟨⬜🟨🟨\n🟩⬜🟩🟩🟩\n...
371,302,1515868241259687940,2022-04-18 01:42:06+00:00,Elosteeze,Wordle 302 3/6\n\n⬛⬛⬛⬛🟨\n🟨🟨🟩🟩🟩\n🟩🟩🟩🟩🟩


In [9]:
previous_df = pd.read_csv("D:/study/Data Mining/Project/input/tweets.csv")
previous_df

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
0,210,1482553374591660037,2022-01-16 03:20:43+00:00,bpszebes,Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩
1,210,1482553387937898499,2022-01-16 03:20:46+00:00,cruisecoup,Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩
2,210,1482553422276698113,2022-01-16 03:20:55+00:00,DestroVega,Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩
3,210,1482553436910628866,2022-01-16 03:20:58+00:00,brenmardash,Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩
4,210,1482553445726908420,2022-01-16 03:21:00+00:00,KatieHowse2,Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩
...,...,...,...,...,...
443122,299,1514808385262170113,2022-04-15 03:30:37+00:00,jesslc143,Wordle 299 4/6\n\n🟨⬛🟨⬛🟩\n⬛⬛🟩⬛🟩\n🟩⬛🟨⬛⬛\n🟩🟩🟩🟩🟩
443123,299,1514808456477171715,2022-04-15 03:30:54+00:00,BJP77,Wordle 299 4/6\n\n⬜⬜🟨⬜🟩\n🟨⬜⬜⬜⬜\n🟨🟩🟨⬜🟩\n🟩🟩🟩🟩🟩
443124,299,1514808460868694022,2022-04-15 03:30:55+00:00,mostholyscum,Wordle 299 6/6\n\n⬛⬛⬛⬛⬛\n⬛🟨⬛⬛⬛\n⬛🟩🟨🟨⬛\n🟨🟩⬛🟩🟩\n...
443125,299,1514808498290266120,2022-04-15 03:31:04+00:00,wjameswolf,Wordle 299 5/6\n\n⬜⬜🟨🟨⬜\n⬜⬜⬜⬜⬜\n⬜🟩⬜⬜🟩\n⬜🟩🟩⬜🟩\n...


In [10]:
df = (pd.concat([previous_df,new_tweets_df])
      .drop_duplicates(subset="tweet_id")
      .sort_values(["wordle_id", "tweet_id"])
      .reset_index(drop=True))
df

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
0,210,1482553374591660037,2022-01-16 03:20:43+00:00,bpszebes,Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩
1,210,1482553387937898499,2022-01-16 03:20:46+00:00,cruisecoup,Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩
2,210,1482553422276698113,2022-01-16 03:20:55+00:00,DestroVega,Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩
3,210,1482553436910628866,2022-01-16 03:20:58+00:00,brenmardash,Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩
4,210,1482553445726908420,2022-01-16 03:21:00+00:00,KatieHowse2,Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩
...,...,...,...,...,...
443495,302,1515874377346207744,2022-04-18 02:06:29+00:00,talbs,Wordle 302 4/6\n\n⬜⬜🟨🟨🟨\n⬜🟨🟨🟨🟩\n🟨🟨🟩🟩🟩\n🟩🟩🟩🟩🟩
443496,302,1515874398426779650,2022-04-18 02:06:34+00:00,octavianasr,Wordle 302 5/6\n\n⬛⬛🟨🟨🟨\n🟨🟨🟨⬛⬛\n⬛🟨🟨🟨⬛\n🟨🟨🟩🟩🟩\n...
443497,302,1515874437282910210,2022-04-18 02:06:44+00:00,m3tamonk,Wordle 302 3/6\n\n⬛🟨⬛🟩⬛\n🟨⬛⬛⬛🟩\n🟩🟩🟩🟩🟩
443498,302,1515874474058465282,2022-04-18 02:06:52+00:00,MrCitroen,Wordle 302 3/6\n\n🟩⬜⬜🟨⬜\n🟩⬜🟩🟩🟩\n🟩🟩🟩🟩🟩


In [11]:
df.groupby("wordle_id")["wordle_id"].count()

wordle_id
210    2837
211    2894
212    2399
213    2119
214    2703
       ... 
296    3623
297    3764
298    3780
299    3821
302     373
Name: wordle_id, Length: 89, dtype: int64

In [12]:
if not os.path.exists("updated"):
    os.mkdir("updated")

df.to_csv("updated/tweets.csv", index=False)